In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from flask import Flask, render_template, request, jsonify, redirect, url_for

In [14]:
# Load and preprocess the dataset
df_long = pd.read_csv('oasis_longitudinal.csv')
df_long.drop(df_long.columns[[0, 1, 3, 4, 6]], axis=1, inplace=True)


In [15]:
# Fill missing values and encode categorical data
df_long['SES'].fillna((round(df_long['SES'].mean())), inplace=True)
df_long['MMSE'].fillna((round(df_long['MMSE'].mean())), inplace=True)
df_long['CDR'].fillna((round(df_long['CDR'].mean())), inplace=True)
df_long['eTIV'].fillna((round(df_long['eTIV'].mean())), inplace=True)
df_long['nWBV'].fillna((round(df_long['nWBV'].mean())), inplace=True)
df_long['ASF'].fillna((round(df_long['ASF'].mean())), inplace=True)
df_long['M/F'] = df_long['M/F'].replace('F', 1)
df_long['M/F'] = df_long['M/F'].replace('M', 0)
df_long['Group'] = df_long['Group'].replace('Demented', 1)
df_long['Group'] = df_long['Group'].replace('Nondemented', 0)
df_long['Group'] = df_long['Group'].replace('Converted', 2)

In [16]:
# Prepare the data for training
array = df_long.values
X = array[:, [1, 2, 3, 4, 5, 6, 7, 8, 9]]
Y = array[:, 0]


In [17]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [19]:
# Train the SVC
svm_classifier = SVC(kernel='rbf', random_state=42, probability=True)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_test)

In [20]:
rand_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rand_classifier.fit(X_train, y_train)
y_pred = rand_classifier.predict(X_test)

In [21]:
# Train the KNeighborClassifier
knn_classifier = KNeighborsClassifier(n_neighbors=6, p=2, metric='euclidean')
knn_classifier.fit(X_train, y_train)
y_pred = knn_classifier.predict(X_test)


In [22]:
# Train the GradientBoostingClassifier
grad_classifier = GradientBoostingClassifier(random_state=42)
grad_classifier.fit(X_train, y_train)
y_pred = grad_classifier.predict(X_test)


In [23]:
# Calculate and print accuracy
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[35  0  0]
 [ 0 36  0]
 [ 1  2  1]]


In [24]:
print(accuracy_score(y_test, y_pred))

0.96


In [25]:
# Flask application
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('one.html')

@app.route('/one_set', methods=['GET'])
def one_set():
    global age, EDUC
    name = request.args.get('name', '')
    age = request.args.get('age', '')
    EDUC = request.args.get('yoe', '')
    return render_template('two.html')

@app.route('/two_set', methods=['GET'])
def two_set():
    global SES, MMSE, CDR
    SES = request.args.get('ses', '')
    MMSE = request.args.get('mmse', '')
    CDR = request.args.get('cdr', '')
    return render_template('three.html')

@app.route('/third_set', methods=['GET'])
def third_set():
    global EDUC, SES, MMSE, CDR, age
    gender = request.args.get('gender', '')
    eTIV = request.args.get('etiv', '')
    nWBV = request.args.get('nwbv', '')
    ASF = request.args.get('asf', '')

    # Prepare the input data
    tp = [EDUC, SES, MMSE, CDR, gender, age, eTIV, nWBV, ASF]
    tp = np.array(tp).astype(float).reshape(1, -1)
    
    # Standardize the input data
    tp = scaler.transform(tp)
    
    value = grad_classifier.predict(tp)[0]

    # Render the appropriate result template
    if value == 0:
        return render_template('result2.html')
    else:
        return render_template('result1.html')

@app.route('/home')
def home():
    return render_template('one.html')

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)

 * Running on http://localhost:5000 (Press CTRL+C to quit)
